### variaveis de inicializacao

In [1]:
from pyspark.sql import SparkSession,Row,DataFrame
from delta import configure_spark_with_delta_pip
from delta.tables import DeltaTable
import pyspark.sql.functions as F
import os.path as path
import traceback
import os

builder = SparkSession \
    .builder \
    .appName("football") \
    .master("local[4]") \
    .config("spark.jars.packages", "uk.co.gresearch.spark:spark-extension_2.12:2.11.0-3.5") \
    .config("spark.jars.packages", "org.apache.spark:spark-avro_2.12:3.5.0") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.warehouse.dir", "/Users/eduardoalberto/LoadFile/output/") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.driver.extraClassPath","/Users/eduardoalberto/opt/spark-3.5.3/jars/mysql-connector-j-9.1.0.jar" ) \
    .config("spark.driver.extraClassPath","/Users/eduardoalberto/opt/spark-3.5.3/jars/postgresql-42.7.5.jar" ) \

spark = configure_spark_with_delta_pip(builder).getOrCreate()

sc = spark.sparkContext
spark.sparkContext.setLogLevel("OFF") 
print('PySpark Version :'+spark.version)
print('PySpark Version :'+spark.sparkContext.version)

spark

25/07/07 21:59:16 WARN Utils: Your hostname, MacBook-Pro-de-Eduardo.local resolves to a loopback address: 127.0.0.1; using 192.168.3.180 instead (on interface en0)
25/07/07 21:59:16 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/Users/eduardoalberto/opt/spark-3.5.4/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/eduardoalberto/.ivy2/cache
The jars for the packages stored in: /Users/eduardoalberto/.ivy2/jars
io.delta#delta-spark_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-b9c29bc0-14b6-4497-a3d5-f63cbbc5f065;1.0
	confs: [default]
	found io.delta#delta-spark_2.12;3.1.0 in central
	found io.delta#delta-storage;3.1.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 245ms :: artifacts dl 12ms
	:: modules in use:
	io.delta#delta-spark_2.12;3.1.0 from central in [default]
	io.delta#delta-storage;3.1.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default

PySpark Version :3.5.4
PySpark Version :3.5.4


In [19]:

df = spark.read.csv("/Users/eduardoalberto/LoadFile/input/BR-Football-Dataset.csv", header=True, inferSchema=True)

df.groupBy(F.col("tournament"),F.col("home"))\
    .agg(F.count_distinct("date").alias("total")).toPandas()



,tournament,home,total
0,Carioca A1,Boavista RJ,9
1,Copa Verde,Luverdense,12
2,Copa do Brasil,Imperatriz,2
3,Copa do Brasil,Ferroviaria SP,7
4,Copa do Brasil,Castanhal,2
...,...,...,...
705,Paulista A2,Sertaozinho FC,5
706,Copa do Brasil,Botafogo PB,7
707,Paulista A3,Capivariano SP,50
708,Copa do Brasil,ABC,14


In [15]:
df.toPandas()

,tournament,home,home_goal,away_goal,away,home_corner,away_corner,home_attack,away_attack,home_shots,away_shots,time,date,ht_diff,at_diff,ht_result,at_result,total_corners
0,Copa do Brasil,Vasco Da Gama RJ,0.0,0.0,ABC,6.0,2.0,133.0,82.0,9.0,6.0,2025-06-19 00:30:00,2023-03-17,0.0,0.0,DRAW,DRAW,8.0
1,Copa do Brasil,Gremio,3.0,0.0,Ferroviario,10.0,3.0,98.0,81.0,30.0,11.0,2025-06-19 23:00:00,2023-03-16,3.0,-3.0,WON,LOST,13.0
2,Copa do Brasil,CSA,1.0,0.0,Brusque,4.0,4.0,105.0,136.0,10.0,13.0,2025-06-19 22:00:00,2023-03-16,1.0,-1.0,WON,LOST,8.0
3,Copa do Brasil,Ypiranga RS,3.0,1.0,Bragantino,7.0,11.0,103.0,119.0,7.0,18.0,2025-06-19 00:30:00,2023-03-16,2.0,-2.0,WON,LOST,18.0
4,Copa do Brasil,Botafogo RJ,7.0,1.0,Brasiliense,8.0,5.0,107.0,121.0,14.0,15.0,2025-06-19 23:00:00,2023-03-15,6.0,-6.0,WON,LOST,13.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14271,Gaucho A1,Veranopolis,1.0,1.0,Internacional RS,3.0,5.0,70.0,128.0,11.0,13.0,2025-06-19 19:00:00,2017-01-29,0.0,0.0,DRAW,DRAW,8.0
14272,Gaucho A1,Internacional RS,3.0,0.0,EC Juventude,5.0,3.0,NaN,NaN,NaN,NaN,2025-06-19 21:02:50,2016-05-08,3.0,-3.0,WON,LOST,8.0
14273,Gaucho A1,EC Juventude,0.0,1.0,Internacional RS,6.0,8.0,NaN,NaN,NaN,NaN,2025-06-19 20:00:00,2016-05-01,-1.0,1.0,LOST,WON,14.0
14274,Gaucho A1,Gremio RS,3.0,1.0,EC Juventude,13.0,0.0,NaN,NaN,NaN,NaN,2025-06-19 20:00:00,2016-04-24,2.0,-2.0,WON,LOST,13.0


### monta script


In [9]:
CONF = {'inptFile': '/Users/eduardoalberto/LoadFile/input/BR-Football-Dataset.csv',
        'path_delta': '/Users/eduardoalberto/LoadFile/output/Football',
        'JDBC':  "jdbc:postgresql://localhost:5432/dev5114",
        'TABLE': "futebol",
        'DB_USER': 'postgres',
        'DB_PASSWORD': 'postgre123'
    }
try:
    if path.isfile(CONF["inptFile"]):
        print("#"*20+" inicio processamento "+"#"*20)
        df_futeboll = spark.read.csv(CONF["inptFile"], header=True,inferSchema=True)\
                                .withColumn('dt_ref_carga',F.expr("current_date()"))

        df_futeboll.write.format("delta")\
                .option("overwriteSchema", "true")\
                .option("path",CONF["path_delta"])\
                .partitionBy("dt_ref_carga")\
                .mode("overwrite")\
                .saveAsTable("default.tb_futebol")
        # postgreSQL
        df_futeboll.write.mode("overwrite").jdbc(CONF["JDBC"], CONF["TABLE"],properties={"user": CONF["DB_USER"],"password": CONF["DB_PASSWORD"],"driver": "org.postgresql.Driver"})
        qtd = df_futeboll.count()
        print("Total de registros = " +str(qtd))
        print("#"*20+" Final processamento "+"#"*20)


    else:
        print("arquivo não existe!")
except Exception as e:
    print(f"Ocorreu o seguinte erro: {e}")
    traceback.print_exc()


#################### inicio processamento ####################


Total de registros = 14276
#################### Final processamento ####################
